## 16.pytorch神经网络基础

### 模型构造：层和块
#### 多层感知机

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)       # 输入的第一个维度，同样还是batch维
print(X)
net(X)

tensor([[0.5175, 0.1732, 0.8512, 0.8381, 0.5448, 0.8256, 0.1194, 0.6489, 0.2193,
         0.8980, 0.4596, 0.8320, 0.1020, 0.3465, 0.7953, 0.5723, 0.9298, 0.9319,
         0.0987, 0.2856],
        [0.2775, 0.9765, 0.5425, 0.2138, 0.1589, 0.4008, 0.9799, 0.8888, 0.3543,
         0.3927, 0.3271, 0.7939, 0.4276, 0.9852, 0.0830, 0.2145, 0.8860, 0.9563,
         0.9683, 0.8577]])


tensor([[-0.2688, -0.1244,  0.0865, -0.2033,  0.0247, -0.0149,  0.1657, -0.2272,
          0.2499,  0.0331],
        [-0.3969, -0.1346,  0.0112, -0.2184,  0.0828,  0.0018,  0.1726, -0.2197,
          0.0602, -0.0211]], grad_fn=<AddmmBackward0>)

#### 自定义module的过程

In [5]:
class MLP(nn.Module):
    # 会继承一些好用的函数比如backward
    def __init__(self):
        # 1.调用父类的初始化
        super().__init__()
        # 2.定义我们需要哪些函数
        self.hidden = nn.Linear(20, 256)        # 线性层，输入20输出256
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        '''定义前向计算过程'''
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[-0.1846, -0.0425, -0.0554, -0.0921, -0.0746,  0.2927,  0.2423,  0.1171,
         -0.2417, -0.0370],
        [-0.1424,  0.1910,  0.0593, -0.0455, -0.1061,  0.3382,  0.2765,  0.3321,
         -0.2208, -0.0991]], grad_fn=<AddmmBackward0>)

#### 顺序块

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block        # 按序的字典，pytorch会知道里面是我们需要的那些层
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.3094, -0.0311, -0.1410, -0.2803, -0.0132,  0.2654, -0.1254, -0.0846,
          0.0162,  0.0166],
        [ 0.2975,  0.0189, -0.1170, -0.0953,  0.0987,  0.1663, -0.1277, -0.1232,
          0.0694,  0.1193]], grad_fn=<AddmmBackward0>)

自定义module的好处在于:

   - 我们能在init和forward的过程中执行大量自定义计算
   - 重写父类的一些方法，可定制性更高

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)    # 随机的、不参与训练的weights
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)                   # X做完linear后，与rand_weight做矩阵乘
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0197, grad_fn=<SumBackward0>)

#### 灵活地嵌套使用

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1347, grad_fn=<SumBackward0>)